
#Efficient Net Modified model for Skin Cancer prediction from an image
In this firs try I will just use the Xception model without the top  and add it a GlobalAveragePooling2D layer and two dense layers, the last one with 3 neurons to represent each one of the possible results:
* Melanoma
* Nevus
* Seborrheic Keratosis

## Differences with 1.8
* In this version we use a same set up as in 1.8 but with 20 epochs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:

from keras.applications import EfficientNetV2L
from keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.applications.efficientnet_v2 import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import Input, Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from keras.models import Model



In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_ds = train_gen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/ZoomCamp/Projects/Capstone 1/ISIC-images/train',
    target_size=(400, 533),
    batch_size=32
)

Found 600 images belonging to 3 classes.


In [ ]:
train_ds.class_indices

{'melanoma': 0, 'nevus': 1, 'seborrheic_keratosis': 2}

In [ ]:
X, y = next(train_ds)

In [ ]:
y[:5]

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/ZoomCamp/Projects/Capstone 1/ISIC-images/val',
    target_size=(400, 533),
    batch_size=32,
    shuffle=False
)

Found 150 images belonging to 3 classes.


In [ ]:
def make_model(learning_rate=0.01, size_inner1=10000, size_inner2=300, droprate=0.5):
    # Use EfficientNetV2L as the base model
    base_model = EfficientNetV2L(
        weights='imagenet',
        include_top=False,
        input_shape=(400, 400, 3)  # Adjust the input shape
    )

    base_model.trainable = False

    #########################################

    inputs = Input(shape=(400, 400, 3))
    base = base_model(inputs, training=False)
    vectors = GlobalAveragePooling2D()(base)
    '''
    inner1 = Dense(size_inner1, activation='relu')(vectors)
    inner1 = BatchNormalization()(inner1)
    drop1 = Dropout(droprate)(inner1)
    '''
    inner2 = Dense(size_inner2, activation='relu')(vectors)
    ##inner2 = BatchNormalization()(inner2)
    drop2 = Dropout(droprate)(inner2)

    outputs = Dense(3)(drop2)

    model = Model(inputs, outputs)

    #########################################

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )

    return model

In [ ]:
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Check TensorFlow GPU devices
print("TensorFlow GPU devices:", tf.config.experimental.list_physical_devices('GPU'))

Num GPUs Available:  0
TensorFlow GPU devices: []


In [ ]:
scores={}
tf.debugging.set_log_device_placement(True)
model = make_model()
history = model.fit(train_ds, epochs=20, validation_data=val_ds)
scores = history.history

Streaming output truncated to the last 5000 lines.
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StatelessTruncatedNormalV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhos

In [ ]:
from pathlib import Path

def save_model_as_h5(model, directory_path, model_name):
    """
    Save a TensorFlow model as an HDF5 file (.h5).

    Parameters:
    - model: The TensorFlow model to be saved.
    - directory_path: The path to the directory where the model file will be saved.
    - model_name: The name of the model file (without extension).

    Example:
    save_model_as_h5(my_model, '/path/to/save', 'my_model')
    """
    # Create the directory if it doesn't exist
    Path(directory_path).mkdir(parents=True, exist_ok=True)

    # Construct the full path to save the model
    model_path = Path(directory_path) / f"{model_name}.h5"

    # Save the model
    model.save(model_path)

    print(f"Model saved to: {model_path}")

In [ ]:
save_model_as_h5(model, '/content/drive/MyDrive/Colab Notebooks/ZoomCamp/Projects/Capstone 1', "capstone1-EN1.0")

NameError: ignored

In [ ]:
plt.plot(scores['accuracy'], label='train')
plt.plot(scores['val_accuracy'], label='val')
plt.xticks(np.arange(20))
plt.legend()

NameError: ignored

## Conclusions
The model training was not able to be concluded before 12 hours that is the free tier and on that time did not show betters results than inception. Due to xception being a lighter and faster model, it is preferible over this one.